In [1]:
# [1] imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
# [4] Hyperparameters
INPUT_SIZE = 28
SEQUENCE_LENGHT = 28
NUM_LAYERS = 2
HIDDEN_SIZE = 256
NUM_CLASSES = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 5

In [3]:
# [3] Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(device)

In [4]:
# [2] Create GRU
class RNN_GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * SEQUENCE_LENGHT, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out
        
    
'''
# Basic model Checks
model = RNN_GRU(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, num_classes=NUM_CLASSES).to(device)
print(model)
'''

'\n# Basic model Checks\nmodel = RNN_GRU(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, num_classes=NUM_CLASSES).to(device)\nprint(model)\n'

In [5]:
# [5] Load data
# Training dataset and loader
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# Testing dataset and loader
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
# [6] Initialize network
model = RNN_GRU(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, num_classes=NUM_CLASSES).to(device)

In [7]:
# [7] Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [8]:
# [8] Train Network
for epoch in range(NUM_EPOCHS):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # remove axes one
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss =  criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

In [9]:
# [9] Check Accuracy on training and test to see how good is my model
def chackAccuracy(loader, model):
        if loader.dataset.train:
            print("Checking Accuracy on TRAIN data")
        else:
            print("Checking Accuracy on TEST data")
        num_correct = 0
        num_samples = 0
        model.eval()

        with torch.no_grad():
            for x,y in loader:
                x = x.to(device).squeeze(1)
                y = y.to(device)

                scores = model(x)
                _, predictions = scores.max(1)
                num_correct += (predictions == y).sum()
                num_samples += predictions.size(0)
            print(f'Got {num_correct} / {num_samples} wich accuracy {float(num_correct)/float(num_samples)*100:.2f}')

        model.train()

chackAccuracy(train_loader, model)
chackAccuracy(test_loader, model)

Checking Accuracy on TRAIN data
Got 59492 / 60000 wich accuracy 99.15
Checking Accuracy on TEST data
Got 9873 / 10000 wich accuracy 98.73
